# 🧠 LangChain RAG Chatbot with Gradio
Upload your document (.txt, .csv, .pdf) and ask natural language questions.

In [ ]:
!pip install gradio langchain langchain_community faiss-cpu openai tiktoken chromadb unstructured sentence-transformers

In [ ]:
import gradio as gr
from tempfile import NamedTemporaryFile
from langchain_community.document_loaders import TextLoader, CSVLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOpenAI
import os

qa_chain = None
chat_history = []

def process_file(file):
    global qa_chain, chat_history
    chat_history = []
    with NamedTemporaryFile(delete=False, suffix=file.name.split('.')[-1]) as tmp:
        tmp.write(file.read())
        path = tmp.name

    if path.endswith(".txt"):
        documents = TextLoader(path).load()
    elif path.endswith(".csv"):
        documents = CSVLoader(file_path=path).load()
    elif path.endswith(".pdf"):
        documents = PyPDFLoader(path).load()
    else:
        return "Unsupported file format"

    chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    retriever = vectorstore.as_retriever()

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return "✅ Document processed!"

In [ ]:
def chat_with_bot(messages):
    if qa_chain is None:
        return messages + [{"role": "assistant", "content": "❌ Please upload a document first."}]
    query = messages[-1]["content"]
    response = qa_chain.run(query)
    return messages + [{"role": "assistant", "content": response}]

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 RAG Chatbot with Document Upload")
    file_input = gr.File(label="Upload a .txt, .csv, or .pdf")
    status = gr.Textbox(label="Status")
    file_input.change(fn=process_file, inputs=file_input, outputs=status)
    gr.ChatInterface(fn=chat_with_bot, title="LangChain RAG Chatbot", chatbot=gr.Chatbot(type="messages"))
demo.launch(share=True)